You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Players, and Impact

Introduction:
- Brief overview of Artificial Intelligence (AI) and its rapid evolution in various industries
- Mention of the latest trends and advancements in AI technology
- Explanation of the importance of staying informed about AI developments

Key Points:
1. Latest Trends in AI:
- Machine learning and deep learning advancements
- Natural language processing and conversational AI
- AI ethics and responsible AI development
- AI-pow

I now can give a great answer

Final Answer:

# The Future of Artificial Intelligence: Trends, Players, and Impact

## Introduction to AI

Artificial Intelligence (AI) has rapidly evolved over the years, transforming various industries and shaping the future of technology. From machine learning and deep learning advancements to natural language processing and conversational AI, the latest trends in AI are revolutionizing how we interact with technology. Staying informed about AI developments is crucial to understanding its impact on society and the ethical considerations surrounding its development.

## Latest Trends in AI

Machine learning and deep learning are at the forefront of AI advancements, allowing machines to learn from data and improve their performance over time. Natural language processing and conversational AI are enabling more seamless interactions between humans and machines, making AI more accessible and user-friendly. The focus on AI ethics and responsible AI developm

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and Impact

## Introduction to AI

Artificial Intelligence (AI) has rapidly evolved over the years, transforming various industries and shaping the future of technology. From machine learning and deep learning advancements to natural language processing and conversational AI, the latest trends in AI are revolutionizing how we interact with technology. Staying informed about AI developments is crucial to understanding its impact on society and the ethical considerations surrounding its development.

## Latest Trends in AI

Machine learning and deep learning are at the forefront of AI advancements, allowing machines to learn from data and improve their performance over time. Natural language processing and conversational AI are enabling more seamless interactions between humans and machines, making AI more accessible and user-friendly. The focus on AI ethics and responsible AI development is gaining traction, emphasizing the importance of ethical considerations in creating AI systems. AI-powered automation is also making a significant impact on industries such as healthcare, finance, and retail, streamlining processes and increasing efficiency.

## Key Players in the AI Industry

Major tech companies like Google, Microsoft, and IBM are leading the way in AI research and development, pushing the boundaries of what AI can achieve. Startups are also disrupting traditional industries with innovative AI solutions, bringing new perspectives and approaches to the table. Academic institutions play a crucial role in contributing to AI innovation, fostering collaboration and knowledge-sharing in the AI community.

## Noteworthy News in AI

Recent breakthroughs in AI research are paving the way for new possibilities and applications in various sectors. AI applications are making a significant impact on society, from improving healthcare outcomes to enhancing customer experiences. Regulatory developments are also shaping the future of AI, ensuring that AI technologies are developed and deployed responsibly.

## Impact of AI on Society

The impact of AI on society is profound, influencing how we work, communicate, and live. As AI continues to advance, it is essential to consider its implications on jobs, privacy, and social interactions. Understanding the potential benefits and challenges of AI is crucial for navigating the future landscape of technology.

## Ethical Considerations in AI Development

Ethical considerations in AI development are paramount, as AI technologies have the power to shape society in significant ways. From bias in AI algorithms to data privacy concerns, addressing ethical issues in AI is critical for building trust and ensuring responsible AI deployment. By prioritizing ethical considerations, we can create AI systems that benefit society as a whole.

## Call to Action: Stay Informed and Engage with AI Community

To stay informed about the latest trends and developments in AI, it is essential to engage with the AI community, participate in discussions, and collaborate on ethical AI practices. By staying informed and actively engaging with AI stakeholders, we can shape the future of AI in a responsible and impactful way.

In conclusion, Artificial Intelligence is a dynamic and evolving field that holds immense potential for transforming industries and society as a whole. By staying informed about the latest trends, key players, and impact of AI, we can navigate the complex landscape of AI development and make informed decisions about its future. Let's embrace the opportunities that AI presents while also addressing the ethical considerations that come with it.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "SpaDeX mission of ISRO"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on SpaDeX mission of ISRO.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: Unveiling the SpaDeX Mission of ISRO: Exploring the Latest Trends and Key Players

Introduction:
- Brief overview of the SpaDeX mission by ISRO
- Importance of space exploration and satellite deployment for data exchange and communication
- Mention of key players involved in the mission

Key Points:
1. Latest Trends:
- Advancements in satellite technology for data exchange and communication
- Increased focus on space exploration and research
- Collaborations between ISRO and int

I now can give a great answer

Final Answer:

# Unveiling the SpaDeX Mission of ISRO: Exploring the Latest Trends and Key Players

## Introduction

The SpaDeX mission by the Indian Space Research Organisation (ISRO) is a groundbreaking endeavor that aims to push the boundaries of space exploration and satellite technology. In an era where data exchange and communication are crucial for various industries, the SpaDeX mission plays a vital role in advancing these capabilities through the deployment of innovative satellites. This mission not only showcases India's prowess in space technology but also highlights the importance of collaboration between international space agencies to achieve common goals.

## Latest Trends

In recent years, there have been significant advancements in satellite technology, enabling faster and more reliable data exchange and communication. The SpaDeX mission is at the forefront of these trends, leveraging cutting-edge technology to enhance the capabilities of

In [14]:
Markdown(result)

# Unveiling the SpaDeX Mission of ISRO: Exploring the Latest Trends and Key Players

## Introduction

The SpaDeX mission by the Indian Space Research Organisation (ISRO) is a groundbreaking endeavor that aims to push the boundaries of space exploration and satellite technology. In an era where data exchange and communication are crucial for various industries, the SpaDeX mission plays a vital role in advancing these capabilities through the deployment of innovative satellites. This mission not only showcases India's prowess in space technology but also highlights the importance of collaboration between international space agencies to achieve common goals.

## Latest Trends

In recent years, there have been significant advancements in satellite technology, enabling faster and more reliable data exchange and communication. The SpaDeX mission is at the forefront of these trends, leveraging cutting-edge technology to enhance the capabilities of satellites in orbit. Moreover, there has been an increased focus on space exploration and research, with a growing number of countries investing in their space programs. Collaborations between ISRO and international space agencies have become more prevalent, with joint missions like SpaDeX paving the way for future cooperation in space exploration.

## Key Players

The key players involved in the SpaDeX mission include the Indian Space Research Organisation (ISRO), known for its expertise in satellite technology and space exploration. Additionally, international space agencies such as the National Aeronautics and Space Administration (NASA) and the European Space Agency (ESA) play crucial roles in supporting and collaborating on the SpaDeX mission. These partnerships highlight the global nature of space exploration and the importance of working together to achieve common objectives.

## Noteworthy News

Recent successful satellite launches by ISRO for the SpaDeX mission have garnered attention worldwide, showcasing India's capabilities in space technology. Updates on the development and deployment of SpaDeX satellites continue to demonstrate the progress made by the mission team in achieving their goals. The achievements and milestones reached by the SpaDeX mission team serve as a testament to the dedication and expertise of all involved in this groundbreaking mission.

In conclusion, the SpaDeX mission by ISRO represents a significant step forward in space exploration and satellite technology. By staying informed and engaged with the latest trends and key players in the field, space enthusiasts, technology professionals, students, and researchers can continue to support and participate in the exciting developments taking place in the world of space exploration. Stay updated on the SpaDeX mission by following official ISRO channels and reputable space news websites, and join the conversation on the future of satellite technology and space research.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).